In [1]:
import pandas as pd
import numpy as np

In [2]:
# read in occupation data
df = pd.read_excel('Full_Occupation_Data.xlsx')

# select only the columns we want
df = df[['AREA', 'AREA_TITLE', 'OCC_CODE', 'OCC_TITLE', 'TOT_EMP', 'A_MEAN']]

# rename columns
df.columns = ['AREA', 'AREA_TITLE', 'OCC_CODE', 'OCC_TITLE', 'TOTAL_EMPLOYED', 'ANNUAL_MEAN_INCOME']

In [3]:
# filter to only counseling occupations
occ_list = ['21-1012', '21-1013', '21-1015', '21-1018', '21-1019', '21-1022', '21-1023', '21-1029']
df = df[df['OCC_CODE'].isin(occ_list)]

In [4]:
# remove na values
df = df.dropna()

In [5]:
# change data types
df['AREA'] = df['AREA'].astype(str)
df['ANNUAL_MEAN_INCOME'] = pd.to_numeric(df['ANNUAL_MEAN_INCOME'], errors='coerce')
df['TOTAL_EMPLOYED'] = pd.to_numeric(df['TOTAL_EMPLOYED'], errors='coerce')
df = df.dropna()
df.dtypes

AREA                   object
AREA_TITLE             object
OCC_CODE               object
OCC_TITLE              object
TOTAL_EMPLOYED        float64
ANNUAL_MEAN_INCOME    float64
dtype: object

In [208]:
# reshape the data frame to be the mean income for all counselors
occupation_df = pd.DataFrame(df.groupby('AREA').agg({'ANNUAL_MEAN_INCOME':'mean', 'TOTAL_EMPLOYED':'sum'}))
occupation_df = occupation_df.reset_index()

In [209]:
# import health data
health_df = pd.read_csv('Full_Health_Data_By_County.csv')

# filter to data about poor mental health
health_df = health_df[health_df['Measure'] == 'Mental health not good for >=14 days among adults aged >=18 years']

In [213]:
# select only the relevant columns
health_df = health_df[['Data_Value', 'TotalPopulation', 'LocationID']]

# rename columns
health_df.columns = ['Percent of Poor Mental Health', 'Total Population', 'FIPS']

In [218]:
# change datatypes
health_df['FIPS'] = health_df['FIPS'].astype(str)

In [221]:
# ensure that all area codes fit the standard 5 digits by adding leading zeros
health_df['FIPS'] = health_df['FIPS'].str.zfill(5)

In [222]:
health_df.head()

,Percent of Poor Mental Health,Total Population,FIPS
316,18.7,47647,22087
382,14.7,122530,20045
541,17.9,152439,22073
701,15.7,16180,08003
704,12.5,3166857,06059


In [232]:
# import data needed to translate area codes
codes_df = pd.read_excel('CBSA_and_FIPS_codes.xls', dtype={'FIPS State Code':'str', 'FIPS County Code':'str'})

# create unified FIPS code
codes_df['FIPS'] = codes_df['FIPS State Code'] + codes_df['FIPS County Code']

In [238]:
# filter to the only two columns we need
codes_df = codes_df[['CBSA Code', 'FIPS']]

In [242]:
# merge the merge the CBSA codes onto the mental health dataframe
health_df = health_df.merge(codes_df, on='FIPS', how='left').dropna()

In [247]:
# create a measure of the total number of people in poor mental health
health_df['Total Poor Mental Health'] = health_df['Percent of Poor Mental Health'] * health_df['Total Population'] / 100

In [254]:
# group by CBSA regions
health_df = pd.DataFrame(health_df.groupby('CBSA Code')[['Total Population', 'Total Poor Mental Health']].sum())
health_df = health_df.reset_index()

In [263]:
health_df.head()

,CBSA Code,Total Population,Total Poor Mental Health
0,10100,85110,8967.654
1,10140,151900,24987.550
2,10180,346370,55479.655
3,10220,76794,13246.965
4,10300,195616,32472.256


In [264]:
occupation_df.head()

,AREA,ANNUAL_MEAN_INCOME,TOTAL_EMPLOYED
0,10180,49235.000000,590.0
1,10380,32115.000000,110.0
2,10420,52688.571429,2680.0
3,10500,49563.333333,270.0
4,10540,64560.000000,290.0


In [270]:
# merge  the CBSA codes the health and occupation data frames
df = occupation_df.merge(health_df, left_on='AREA', right_on='CBSA Code', how='left').dropna()

In [278]:
# select only the columns we want
df = df[['ANNUAL_MEAN_INCOME', 'TOTAL_EMPLOYED', 'CBSA Code',
       'Total Population', 'Total Poor Mental Health']]

# rename columns
df.columns = ['mean_income', 'total_employed', 'cbsa', 'population', 'poor_mental_health']

In [279]:
df.head()

,mean_income,total_employed,cbsa,population,poor_mental_health
0,49235.000000,590.0,10180,346370.0,55479.655
2,52688.571429,2680.0,10420,1402898.0,228244.974
3,49563.333333,270.0,10500,296354.0,50270.225
4,64560.000000,290.0,10540,262108.0,41150.956
5,59085.000000,4010.0,10580,1757100.0,237785.436


In [280]:
# save to file
df.to_csv('distress_vs_resources.csv')